# Arbitrarily high order accurate explicit time integration methods

 1. Chapter 5: ADER and DeC
    1. [Section 1.1: DeC](#DeC)
    1. [Section 1.2: ADER](#ADER)

## Deferred Correction (Defect correction/ Spectral deferred correction)<a id='DeC'></a>
Acronyms: DeC, DEC, DC, SDC

References: [Dutt et al. 2000](https://link.springer.com/article/10.1023/A:1022338906936), [Minion (implicit) 2003](https://projecteuclid.org/journals/communications-in-mathematical-sciences/volume-1/issue-3/Semi-implicit-spectral-deferred-correction-methods-for-ordinary-differential-equations/cms/1250880097.full), [Abgrall 2017 (for PDE)](https://hal.archives-ouvertes.fr/hal-01445543v2)

We study Abgrall's version (for notation)

Theory on slides!

In [ ]:
# Loading/installing packages

# This is the basic package in python with all the numerical functions
try:
    import numpy as np
except ImportError:
    %pip install numpy
    import numpy as np

# This package allows to  plot
try:
    import matplotlib.pyplot as plt 
except ImportError:
    %pip install matplotlib
    import matplotlib.pyplot as plt 

#This package already implemented some functions for Runge Kutta and multistep methods
try:
    from nodepy import rk
except ImportError:
    %pip install nodepy
    from nodepy import rk

In [ ]:
# Download collection of ODE problems
try:
    from ODEproblems import ODEproblem
except ImportError:
    ![ -f ODEproblems.py ] || wget https://github.com/accdavlo/HighOrderODESolvers/raw/master/ODEproblems.py -O ODEproblems.py
    from ODEproblems import ODEproblem

For the definition of the basis functions in time, we introduce different Lagrange polynomials and point distributions:
1. equispaced
1. Gauss--Legendre--Lobatto (GLB)
1. Gauss--Legendre (not in DeC, because the last point is not $t^{n+1}$)

So, we have the quadrature points $\lbrace t^m \rbrace_{m=0}^M$, the polynomials $\lbrace \varphi_m \rbrace_{m=0}^M$ such that $\varphi_j(t^m)=\delta_{j}^m$, and we are interested in computing

$$
\theta_r^m:=\int_{t^0}^{t^m} \varphi_r(t) dt
$$

To compute the integral we will use exact quadrature rules with Gauss--Lobatto (GLB) points, i.e., given the quadrature nodes and weights $t_q, w_q$ on the interval $[0,1]$ the integral is computed as

$$
\theta_r^m:=\int_{t^0}^{t^m} \varphi_r(t) dt = \sum_q \varphi_r(t^q(t^m-t^0)+t^0) w_q(t^m-t^0) 
$$


In practice, at each timestep we have to loop over corrections $(k)$ and over subtimesteps $m$ and compute

$$
y^{m,(k)} = y^{m,(k-1)} - \left(  y^{m,(k-1)} -  y^{0} - \Delta t\sum_{r=0}^M \theta_r^m F(y^{r,(k-1)}) \right)=y^{0} + \Delta t\sum_{r=0}^M \theta_r^m F(y^{r,(k-1)})
$$

In [ ]:
from scipy.interpolate import lagrange
from numpy.polynomial.legendre import leggauss

def equispaced(order):
    '''
    Takes input d and returns the vector of d equispaced points in [-1,1]
    And the integral of the basis functions interpolated in those points
    '''
    nodes= np.linspace(-1,1,order)
    w= np.zeros(order)
    for k in range(order):
        yy= np.zeros(order)
        yy[k]=1.
        zz=lagrange(nodes,yy)
        pp=zz.integ()
        w[k]=pp(1)-pp(-1)

    return nodes, w

def lglnodes(n,eps=10**-15):
    '''
    Python translation of lglnodes.m

    Computes the Legendre-Gauss-Lobatto nodes, weights and the LGL Vandermonde 
    matrix. The LGL nodes are the zeros of (1-x^2)*P'_N(x). Useful for numerical
    integration and spectral methods. 

    Parameters
    ----------
    n : integer, requesting an nth-order Gauss-quadrature rule on [-1, 1]

    Returns
    -------
    (nodes, weights) : tuple, representing the quadrature nodes and weights.
                       Note: (n+1) nodes and weights are returned.
            

    Example
    -------
    >>> from lglnodes import *
    >>> (nodes, weights) = lglnodes(3)
    >>> print(str(nodes) + "   " + str(weights))
    [-1.        -0.4472136  0.4472136  1.       ]   [0.16666667 0.83333333 0.83333333 0.16666667]

    Notes
    -----

    Reference on LGL nodes and weights:  
      C. Canuto, M. Y. Hussaini, A. Quarteroni, T. A. Tang, "Spectral Methods
      in Fluid Dynamics," Section 2.3. Springer-Verlag 1987

    Written by Greg von Winckel - 04/17/2004
        Contact: gregvw@chtm.unm.edu

    Translated and modified into Python by Jacob Schroder - 9/15/2018 
    '''

    w = np.zeros((n+1,))
    x = np.zeros((n+1,))
    xold = np.zeros((n+1,))

    # The Legendre Vandermonde Matrix
    P = np.zeros((n+1,n+1))

    epss = eps

    # Use the Chebyshev-Gauss-Lobatto nodes as the first guess
    for i in range(n+1): 
        x[i] = -np.cos(np.pi*i / n)
  
  
    # Compute P using the recursion relation
    # Compute its first and second derivatives and 
    # update x using the Newton-Raphson method.
    
    xold = 2.0
    
    for i in range(100):
        xold = x
       
        P[:,0] = 1.0 
        P[:,1] = x
       
        for k in range(2,n+1):
            P[:,k] = ( (2*k-1)*x*P[:,k-1] - (k-1)*P[:,k-2] ) / k
       
        x = xold - ( x*P[:,n] - P[:,n-1] )/( (n+1)*P[:,n]) 
        
        if (max(abs(x - xold).flatten()) < epss ):
            break 
    
    w = 2.0 / ( (n*(n+1))*(P[:,n]**2))
    
    return x, w
 
def lagrange_basis(nodes,x,k):
    """ Lagrange basis functions
    Input:
    nodes: (array) set of nodes that defines the polynomials
    x: (array)     points where to evaluate the function
    k: (int)       number of Lagrangian basis function to evaluate
    """
    y=np.zeros(x.size)
    for ix, xi in enumerate(x):
        tmp=[(xi-nodes[j])/(nodes[k]-nodes[j])  for j in range(len(nodes)) if j!=k]
        y[ix]=np.prod(tmp)
    return y

def get_nodes(order,nodes_type):
    """ Obtain nodes for different node distributions in [0,1]
    Input:
    order: (int)  number of points
    nodes_type: (str) nodes distribution ("equispaced","gaussLobatto","gaussLegendre")

    Output:
    nodes (array) points
    w (array)     weights of the relative quadrature formula
    """
    if nodes_type=="equispaced":
        nodes,w = equispaced(order)
    elif nodes_type == "gaussLegendre":
        nodes,w = leggauss(order)
    elif nodes_type == "gaussLobatto":
        nodes, w = lglnodes(order-1,10**-15)
    nodes=nodes*0.5+0.5
    w = w*0.5
    return nodes, w
        
def compute_theta_DeC(order, nodes_type):
    """ Computes theta and beta coefficients of dec
    Input:
    order: (int)  number of points
    nodes_type: (str) nodes distribution ("equispaced","gaussLobatto","gaussLegendre")

    Output:
    theta: array (order x order) theta coefficients
    beta: array (order) beta coefficients
    """
    # Polynomial nodes
    nodes, w = get_nodes(order,nodes_type)
    # Quadrature nodes (exact)
    int_nodes, int_w = get_nodes(order,"gaussLobatto")
    # generate theta and beta coefficients 
    theta = np.zeros((order,order))
    beta = np.zeros(order)
    # Loop over coefficients
    for m in range(order):
        beta[m] = nodes[m]
        # Rescaling of quadrature points in the interval [0,beta[m]]
        nodes_m = int_nodes*(beta[m])
        w_m = int_w*(beta[m])
        # Computing integrals of int_0^beta[m] phi_r 
        for r in range(order):
            theta[r,m] = sum(lagrange_basis(nodes,nodes_m,r)*w_m)
    return theta, beta


def compute_RK_from_DeC(M_sub,K_corr,nodes_type):
    """ Compute the RK matrices of an explicit DeC
    Input:
    M_sub: (int)  number of subtimeintervals (M_sub+1 nodes)
    K_corr: (int) number of iterations
    nodes_type: (str) nodes distribution ("equispaced","gaussLobatto","gaussLegendre")

    Output:
    A,b,c (arrays) RK structures    
    """
    order=M_sub+1
    [theta,beta]=compute_theta_DeC(order,nodes_type)
    bar_beta=beta[1:]  # M_sub
    bar_theta=theta[:,1:].transpose() # M_sub x (M_sub +1)
    theta0= bar_theta[:,0]  # M_sub x 1
    bar_theta= bar_theta[:,1:] #M_sub x M_sub
    A=np.zeros((M_sub*(K_corr-1)+1,M_sub*(K_corr-1)+1))  # (M_sub x K_corr +1)^2
    b=np.zeros(M_sub*(K_corr-1)+1)
    c=np.zeros(M_sub*(K_corr-1)+1)

    c[1:M_sub+1]=bar_beta
    A[1:M_sub+1,0]=bar_beta
    for k in range(1,K_corr-1):
        r0=1+M_sub*k
        r1=1+M_sub*(k+1)
        c0=1+M_sub*(k-1)
        c1=1+M_sub*(k)
        c[r0:r1]=bar_beta
        A[r0:r1,0]=theta0
        A[r0:r1,c0:c1]=bar_theta
    b[0]=theta0[-1]
    b[-M_sub:]=bar_theta[M_sub-1,:]
    return A,b,c


In [ ]:
## Deferred correction algorithm

def dec(func, tspan, y_0, M_sub, K_corr, distribution):
    '''
    Deferred correction algorithm with the formalism of Abgrall 2017
    Input:
    func: (lambda function) RHS of the ODE
    tspan: (array (N_time)) the timesteps
    y_0: (array (dim)) is the initial value
    M_sub: (int) number of subtimeintervals [t^0,t^m], m=0,\dots, M_sub
    K_corr: (int) number of iterations of the algorithm (order=K_corr)
    distribution: (str) distribution of the subtimenodes between "equispaced", "gaussLobatto"

    Output:
    tspan (array) timestpes
    U (array)     solutions at the different timesteps
    '''
    N_time=len(tspan)
    dim=len(y_0)
    U=np.zeros((dim, N_time))
    u_p=np.zeros((dim, M_sub+1))
    u_a=np.zeros((dim, M_sub+1))
    rhs= np.zeros((dim,M_sub+1))
    Theta, beta = compute_theta_DeC(M_sub+1,distribution)
    U[:,0]=y_0
    for it in range(1, N_time):
        delta_t=(tspan[it]-tspan[it-1])
        for m in range(M_sub+1):
            u_a[:,m]=U[:,it-1]
            u_p[:,m]=U[:,it-1]
        for k in range(1,K_corr+1):
            u_p=np.copy(u_a)
            for r in range(M_sub+1):
                rhs[:,r]=func(u_p[:,r])
            for m in range(1,M_sub+1):
                u_a[:,m]= U[:,it-1]+delta_t*sum([Theta[r,m]*rhs[:,r] for r in range(M_sub+1)])
        U[:,it]=u_a[:,M_sub]
    return tspan, U

In [ ]:
pr=ODEproblem("threeBodies")
tt=np.linspace(0,pr.T_fin,1000)
tt,U=dec(pr.flux,tt,pr.u0,4,5,"gaussLobatto")
plt.figure()
plt.plot(U[0,:],U[1,:],'*',label="sun")
plt.plot(U[4,:],U[5,:],label="earth")
plt.plot(U[8,:],U[9,:],label="Mars")
plt.legend()
plt.show()

plt.figure()
plt.title("Distance from the original position of the sun")
plt.semilogy(tt,U[4,:]**2+U[5,:]**2,label="earth")
plt.semilogy(tt,U[8,:]**2+U[9,:]**2, label="mars")
plt.legend()
plt.show()

In [ ]:
#Test convergence
pr=ODEproblem("linear_system2")

tt=np.linspace(0,pr.T_fin,10)
tt,uu=dec(pr.flux, tt, pr.u0, 7, 8, "equispaced")
plt.plot(tt,uu[0,:])
plt.plot(tt,uu[1,:])
plt.show()

def compute_integral_error(c,c_exact):  # c is dim x times
    times=np.shape(c)[1]
    error=0.
    for t in range(times):
        error = error + np.linalg.norm(c[:,t]-c_exact[:,t],2)**2.
    error = np.sqrt(error/times) 
    return error

NN=5
dts=[pr.T_fin/2.0**k for k in range(3,3+NN)]
errorsDeC=np.zeros(len(dts))

for order in range(2,10):
    for k in range(NN):
        dt0=dts[k]
        tt=np.arange(0,pr.T_fin,dt0)
        t2,U2=dec(pr.flux, tt, pr.u0, order-1, order, "gaussLobatto")
        u_exact=pr.exact_solution_times(pr.u0,tt)
        errorsDeC[k]=compute_integral_error(U2,u_exact)

    plt.loglog(dts,errorsDeC,"--",label="DeC%d"%(order))
    plt.loglog(dts,[dt**order*errorsDeC[2]/dts[2]**order for dt in dts],":",label="ref %d"%(order))


plt.title("DeC error convergence")
plt.legend()
#plt.savefig("convergence_DeC.pdf")
plt.show()


In [ ]:
for order in range(2,10):
    A,b,c=compute_RK_from_DeC(order-1,order,"equispaced")
    rkDeC = rk.ExplicitRungeKuttaMethod(A,b)
    rkDeC.name="DeC"+str(order)
    rkDeC.plot_stability_region(bounds=[-5,3,-7,7])

In [ ]:
for order in range(2,14):
    A,b,c=compute_RK_from_DeC(order-1,order,"equispaced")
    rkDeC = rk.ExplicitRungeKuttaMethod(A,b)
    rkDeC.name="DeC"+str(order)
    print(rkDeC.name+" has order "+str(rkDeC.order()))

In [ ]:
pr=ODEproblem("lotka")
tt=np.linspace(0,pr.T_fin,150)
t2,U2=dec(pr.flux, tt, pr.u0, 1, 2, "gaussLobatto")
t8,U8=dec(pr.flux, tt, pr.u0, 7, 8, "gaussLobatto")

tt=np.linspace(0,pr.T_fin,2000)
tref,Uref=dec(pr.flux, tt, pr.u0, 4,5, "gaussLobatto")

plt.figure(figsize=(12,6))
plt.subplot(211)
plt.plot(t2,U2[0,:],label="dec2")
plt.plot(t8,U8[0,:],label="dec8")
plt.plot(tref,Uref[0,:], ":",linewidth=2,label="ref")
plt.legend()
plt.title("Prey")

plt.subplot(212)
plt.plot(t2,U2[1,:],label="dec2")
plt.plot(t8,U8[1,:],label="dec8")
plt.plot(tref,Uref[1,:],":", linewidth=2,label="ref")
plt.legend()
plt.title("Predator")

### Pro exercise: implement the implicit DeC presented in the slides (Rosenbrock style)
* You need to pass also a function of the Jacobian of the flux in input
* The Jacobian can be evaluated only once per timestep $\partial_y F(y^n)$ and used to build the matrix that must be inverted at each correction
* For every subtimestep the matrix to be inverted changes a bit ($\beta^m \Delta t$ factor in front of the Jacobian)
* One can invert these $M$ matrices only once per time step
* Solve the system at each subtimestep and iteration

$$
y^{m,(k)}-\beta^m \Delta t \partial_y F(y^0)y^{m,(k)} = y^{m,(k-1)}-\beta^m \Delta t \partial_y F(y^0)y^{m,(k-1)} - \left(  y^{m,(k-1)} -  y^{0} - \Delta t\sum_{r=0}^M \theta_r^m F(y^{r,(k-1)}) \right)
$$
defining $M^{m}=I-\beta^m \Delta t \partial_y F(y^0)$, we can simplify it as 
$$
y^{m,(k)}=y^{m,(k-1)} - (M^m)^{-1}\left(  y^{m,(k-1)} -  y^{0} - \Delta t\sum_{r=0}^M \theta_r^m F(y^{r,(k-1)}) \right)
$$
or
$$
M^m y^{m,(k)}=\left(   y^{0} + \Delta t\sum_{r=0}^M \theta_r^m F(y^{r,(k-1)}) -\beta^m \Delta t \partial_y F(y^0)y^{m,(k-1)} \right)
$$

In [ ]:
def decImplicit(func,jac_stiff, tspan, y_0, M_sub, K_corr, distribution):
    """
    The decImplicit function implements the DeC (Difference-Corrected) method for solving
    initial value problems (IVPs) in ordinary differential equations (ODEs).

    Inputs:
    func: callable      Right-hand side of the ODE system.
    jac_stiff: callable Jacobian matrix of the right-hand side of the ODE system, used for stiffness correction.
    tspan: numpy array  Time steps for the simulation.
    y_0: numpy array    Initial conditions for the ODE system.
    M_sub: int          Number of sub-intervals.
    K_corr: int         Number of correction steps.
    distribution: str   nodes distribution

    Outputs:
    tspan: numpy array  Time steps for the simulation.
    U: numpy array      Numerical solution to the ODE system.

    Note:
    The inputs func and jac_stiff should be function handles that return the right-hand side and
    Jacobian matrix, respectively, for the given input argument(s).
    """
    
    N_time=len(tspan) # Compute the number of time steps
    dim=len(y_0) # Compute the dimension of the ODE system
    U=np.zeros((dim, N_time)) # Initialize the solution array
    
    # Initialize arrays for predictor and corrector
    u_p=np.zeros((dim, M_sub+1))
    u_a=np.zeros((dim, M_sub+1))
    
    # Initialize temporary arrays
    u_help= np.zeros(dim)
    rhs= np.zeros((dim,M_sub+1))
    
    # Compute the coefficients for the DeC method
    Theta, beta = compute_theta_DeC(M_sub+1,distribution)
    
    # Initialize the inverse Jacobian matrix
    invJac=np.zeros((M_sub+1,dim,dim))
    
    # Set the initial conditions
    U[:,0]=y_0
    
    # Loop over each time step
    for it in range(1, N_time):
        # Compute the time step size
        delta_t=(tspan[it]-tspan[it-1])
        
        # Initialize the predictor and corrector arrays
        for m in range(M_sub+1):
            u_a[:,m]=U[:,it-1]
            u_p[:,m]=U[:,it-1]
        
        # Compute the Jacobian matrix at the start of the time step
        SS=jac_stiff(u_p[:,0])
        
        # Compute the inverse Jacobian matrix for each sub-interval
        for m in range(1,M_sub+1):
            invJac[m,:,:]=np.linalg.inv(np.eye(dim) - delta_t*beta[m]*SS)

        # Iterative loop for correction
        for k in range(1, K_corr + 1):
            # Copy the previous solution
            u_p = np.copy(u_a)
            # Compute the right-hand side of the ODE
            for r in range(M_sub + 1):
                rhs[:, r] = func(u_p[:, r])
            # Update the intermediate solution
            for m in range(1, M_sub + 1):
                u_a[:, m] = u_p[:, m] + delta_t * np.matmul(invJac[m, :, :], (
                    -(u_p[:, m] - u_p[:, 0]) / delta_t + sum(
                        [Theta[r, m] * rhs[:, r] for r in range(M_sub + 1)])))
        # Update the solution
        U[:, it] = u_a[:, M_sub]
    return tspan, U

In [ ]:
# Test on Robertson problem
pr=ODEproblem("Robertson")

Nt=100
tt = np.array([np.exp(k) for k in np.linspace(-14,np.log(pr.T_fin),Nt)])
tt,yy=decImplicit(pr.flux,pr.jacobian, tt, pr.u0, 5,6,"gaussLobatto")

plt.semilogx(tt,yy[0,:])
plt.semilogx(tt,yy[1,:]*10**4)
plt.semilogx(tt,yy[2,:])

In [ ]:
Nt=1000
tt = np.array([np.exp(k) for k in np.linspace(-14,np.log(pr.T_fin),Nt)])
tt,yy=dec(pr.flux, tt, pr.u0, 5,6,"gaussLobatto")

plt.semilogx(tt,yy[0,:])
plt.semilogx(tt,yy[1,:]*10**4)
plt.semilogx(tt,yy[2,:])
plt.ylim([-0.05,1.05])

## ADER <a id='ADER'></a>
Can be interpreted as a finite element method in time solved in an iterative manner.

\begin{align*}
		\mathcal{L}^2(\underline{\mathbf{c}} ):=& \int_{T^n} \underline{\phi}(t) \partial_t \underline{\phi}(t)^T \underline{\mathbf{c}} dt + \int_{T^n} \underline{\phi}(t)  F(\underline{\phi}(t)^T\underline{\mathbf{c}})  dt =\\
		&\underline{\phi}(t^{n+1}) \underline{\phi}(t^{n+1})^T \underline{\mathbf{c}} - \underline{\phi}(t^{n}) \boldsymbol{c}^n -  \int_{T^n} \partial_t \underline{\phi}(t) \underline{\phi}(t)^T \underline{\mathbf{c}}   - \int_{T^n} \underline{\phi}(t)  F(\underline{\phi}(t)^T\underline{\mathbf{c}})  dt \\
&\underline{\underline{\mathrm{M}}} = \underline{\phi}(t^{n+1}) \underline{\phi}(t^{n+1})^T -\int_{T^n} \partial_t \underline{\phi}(t) \underline{\phi}(t)^T \\
&	\underline{r}(\underline{\mathbf{c}}) =  \underline{\phi}(t^{n}) \boldsymbol{c}^n + \int_{T^n} \underline{\phi}(t)  F(\underline{\phi}(t)^T\underline{\mathbf{c}})  dt\\ 
&\underline{\underline{\mathrm{M}}} \underline{\mathbf{c}} = \underline{r}(\underline{\mathbf{c}})
\end{align*}

Iterative procedure to solve the problem for each time step

\begin{equation}
\underline{\mathbf{c}}^{(k)}=\underline{\underline{\mathrm{M}}}^{-1}\underline{r}(\underline{\mathbf{c}}^{(k-1)}),\quad k=1,\dots, K \text{ (convergence)}
\end{equation}

with $\underline{\mathbf{c}}^{(0)}=\boldsymbol{c}(t^n)$.

Reconstruction step

\begin{equation*}
	\boldsymbol{c}(t^{n+1}) = \underline{\phi}(t^{n+1})^T \underline{\mathbf{c}}^{(K)}.
\end{equation*}

### What can be precomputed?
* $\underline{\underline{\mathrm{M}}}$
* $$\underline{r}(\underline{\mathbf{c}}) =  \underline{\phi}(t^{n}) \boldsymbol{c}^n + \int_{T^n} \underline{\phi}(t)  F(\underline{\phi}(t)^T\underline{\mathbf{c}})  dt\approx \underline{\phi}(t^{n}) \boldsymbol{c}^n + \int_{T^n} \underline{\phi}(t)\underline{\phi}(t)^T dt  F(\underline{\mathbf{c}}) =  \underline{\phi}(t^{n}) \boldsymbol{c}^n+ \underline{\underline{\mathrm{R}}} \underline{\mathbf{c}}$$ 
$\underline{\underline{\mathrm{R}}}$ can be precomputed
* $$ \boldsymbol{c}(t^{n+1}) = \underline{\phi}(t^{n+1})^T \underline{\mathbf{c}}^{(K)} $$
$\underline{\phi}(t^{n+1})^T$ can be precomputed

In [ ]:
from scipy.interpolate import lagrange

# Define the Lagrange polynomial
def lagrange_poly(nodes, k):
    interpVal = np.zeros(np.size(nodes))
    interpVal[k] = 1.
    pp = lagrange(nodes, interpVal)
    return pp

# Evaluate the Lagrange basis function
def lagrange_basis(nodes, x, k):
    pp = lagrange_poly(nodes, k)
    return pp(x)

# Evaluate the derivative of the Lagrange polynomial
def lagrange_deriv(nodes, x, k):
    pp = lagrange_poly(nodes, k)
    dd = pp.deriv()
    return dd(x)

# Get nodes and weights for different quadrature nodes types
def get_nodes(order, nodes_type):
    if nodes_type == "equispaced":
        nodes, w = equispaced(order)
    elif nodes_type == "gaussLegendre":
        nodes, w = leggauss(order)
    elif nodes_type == "gaussLobatto":
        nodes, w = lglnodes(order - 1, 10**-15)
    nodes = nodes * 0.5 + 0.5
    w = w * 0.5
    return nodes, w

# Get the ADER matrices
def getADER_matrix(order, nodes_type):
    ''' Compute ADER matrices
    Input:
    order: (int) number of nodes
    nodes_type: (str) distribution of nodes

    Output:
    nodes_poly: (array) subtimenodes in [0,1]
    w_poly: (array) weights of the quadrature related to subtimenodes
    M: (array 2d) LHS matrix of ADER algorithm
    RHSmat: (array 2d) RHS matrix of ADER algorithm
    evolMatrix: (array 2d) M^{-1} RHSmat
    reconstructionCoefficients: (array) coefficients to extrapolate in 1
    '''
    nodes_poly, w_poly = get_nodes(order, nodes_type)
    if nodes_type == "equispaced":
        quad_order = order
        nodes_quad, w = get_nodes(quad_order, "gaussLegendre")
    else:
        quad_order = order
        nodes_quad, w = get_nodes(quad_order, nodes_type)

    # Generate mass matrix
    M = np.zeros((order, order))
    for i in range(order):
        for j in range(order):
            M[i, j] = lagrange_basis(nodes_poly, 1.0, i) * lagrange_basis(nodes_poly, 1.0, j)\
                      - sum([lagrange_deriv(nodes_poly, nodes_quad[q], i)\
                      * lagrange_basis(nodes_poly, nodes_quad[q], j)\
                      * w[q] for q in range(quad_order)])
    # Generate RHS matrix
    RHSmat = np.zeros((order, order))
    for i in range(order):
        for j in range(order):
            RHSmat[i, j] = sum([lagrange_basis(nodes_poly, nodes_quad[q], i)\
                                 * lagrange_basis(nodes_poly, nodes_quad[q], j)\
                                 * w[q] for q in range(quad_order)])

    # Evolution matrix
    invader = np.linalg.inv(M)
    evolMatrix=np.matmul(invader,RHSmat)

    # Reconstruction coefficients
    reconstructionCoefficients=np.array([lagrange_basis(nodes_poly,1.0,i) for i in range(order)])
    
    return nodes_poly, w_poly, M, RHSmat, evolMatrix, reconstructionCoefficients

# Solve a system of ODEs using the ADER method
def ader(func, tspan, y_0, M_sub, K_corr, distribution):
    '''
    ADER algorithm for ODEs with the formalism of Han Veiga, Oeffner, Torlo 2021
    Input:
    func: (lambda function) RHS of the ODE
    tspan: (array (N_time)) the timesteps
    y_0: (array (dim)) is the initial value
    M_sub: (int) number of subtimeintervals [t^0,t^m], m=0,\dots, M_sub
    K_corr: (int) number of iterations of the algorithm (order=K_corr)
    distribution: (str) distribution of the subtimenodes between "equispaced", "gaussLobatto", "gaussLegendre"

    Output:
    tspan (array) timestpes
    U (array)     solutions at the different timesteps    
    '''
    
    N_time=len(tspan)
    dim=len(y_0)
    U=np.zeros((dim, N_time))
    u_p=np.zeros((dim, M_sub+1))
    u_a=np.zeros((dim, M_sub+1))
    u_tn=np.zeros((dim, M_sub+1))
    rhs= np.zeros((dim,M_sub+1))
    
    x_poly, w_poly, ADER, RHS_mat, evolMatrix, reconstructionCoefficients = getADER_matrix(M_sub+1, distribution)
    
    U[:,0]=y_0
    
    for it in range(1, N_time):
        delta_t=(tspan[it]-tspan[it-1])
        for m in range(M_sub+1):
            u_a[:,m]=U[:,it-1]
            u_p[:,m]=U[:,it-1]
            u_tn[:,m]=U[:,it-1]
        for k in range(1,K_corr+1):
            u_p=np.copy(u_a)
            for r in range(M_sub+1):
                rhs[:,r]=func(u_p[:,r])
            for d in range(dim):
                u_a[d,:] = u_tn[d,:] + delta_t*np.matmul(evolMatrix,rhs[d,:])
        U[:,it] = np.matmul(u_a,reconstructionCoefficients)
    return tspan, U

In [ ]:
pr=ODEproblem("threeBodies")
tt=np.linspace(0,pr.T_fin,1000)
tt,U=ader(pr.flux,tt,pr.u0,4,5,"gaussLegendre")
plt.figure()
plt.plot(U[0,:],U[1,:],'*',label="sun")
plt.plot(U[4,:],U[5,:],label="earth")
plt.plot(U[8,:],U[9,:],label="Mars")
plt.legend()
plt.show()

plt.figure()
plt.title("Distance from the original position of the sun")
plt.semilogy(tt,U[4,:]**2+U[5,:]**2,label="earth")
plt.semilogy(tt,U[8,:]**2+U[9,:]**2, label="mars")
plt.legend()
plt.show()

In [ ]:
#Test convergence
pr=ODEproblem("linear_system2")

tt=np.linspace(0,pr.T_fin,10)
tt,uu=ader(pr.flux, tt, pr.u0, 7, 8, "equispaced")
plt.plot(tt,uu[0,:])
plt.plot(tt,uu[1,:])
plt.show()

def compute_integral_error(c,c_exact):  # c is dim x times
    times=np.shape(c)[1]
    error=0.
    for t in range(times):
        error = error + np.linalg.norm(c[:,t]-c_exact[:,t],2)**2.
    error = np.sqrt(error/times) 
    return error

NN=5
dts=[pr.T_fin/2.0**k for k in range(3,3+NN)]
errorsDeC=np.zeros(len(dts))

for order in range(2,8):
    for k in range(NN):
        dt0=dts[k]
        tt=np.arange(0,pr.T_fin,dt0)
        t2,U2=ader(pr.flux, tt, pr.u0, order-1, order, "gaussLobatto")
        u_exact=pr.exact_solution_times(pr.u0,tt)
        errorsDeC[k]=compute_integral_error(U2,u_exact)

    plt.loglog(dts,errorsDeC,"--",label="ADER%d"%(order))
    plt.loglog(dts,[dt**order*errorsDeC[2]/dts[2]**order for dt in dts],":",label="ref %d"%(order))


plt.title("ADER error convergence")
plt.legend()
#plt.savefig("convergence_ADER.pdf")
plt.show()


In [ ]:
pr=ODEproblem("lotka")
tt=np.linspace(0,pr.T_fin,150)
t2,U2=ader(pr.flux, tt, pr.u0, 1, 2, "gaussLobatto")
t8,U8=ader(pr.flux, tt, pr.u0, 7, 8, "gaussLobatto")

tt=np.linspace(0,pr.T_fin,2000)
tref,Uref=dec(pr.flux, tt, pr.u0, 4,5, "gaussLobatto")

plt.figure(figsize=(12,6))
plt.subplot(211)
plt.plot(t2,U2[0,:],label="ADER2")
plt.plot(t8,U8[0,:],label="ADER8")
plt.plot(tref,Uref[0,:], ":",linewidth=2,label="ref")
plt.legend()
plt.title("Prey")

plt.subplot(212)
plt.plot(t2,U2[1,:],label="ADER2")
plt.plot(t8,U8[1,:],label="ADER8")
plt.plot(tref,Uref[1,:],":", linewidth=2,label="ref")
plt.legend()
plt.title("Predator")

### Pro exercise: implicit ADER
Using the fact that ADER can be written into DeC, try to make ADER implicit by changing only the definition of $\mathcal{L}^1$

* Write the formulation and the update formula
* Implement it adding (as for the DeC an extra input of the jacobian of the flux)

### Pro exercise: ADER as RK
How can you write the ADER scheme into a RK setting?
At the end we are computing some coefficients in a more elaborated way to use them explicitly, so one should be able to write it down.

#### Stability ADER
**Stability for ADER and DeC methods with $p$ subtimesteps**

| ADER        | ADER vs DeC |
| ----------- | ----------- |
| ![ADERStability](https://github.com/accdavlo/HighOrderODESolvers/raw/master/images/chapter5/stabilityADER.png)  | ![ADERStability](https://github.com/accdavlo/HighOrderODESolvers/raw/master/images/chapter5/stabilityADERvsDeC.png)     |
